In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
import time

In [3]:
tf.config.list_physical_devices('GPU')

[]

### About the mnist data set : http://yann.lecun.com/exdb/mnist/

In [4]:

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

template1 ='whole : {} | lev 1: {} | lev 2: {}'
# print(template1.format(len(x_train), len(x_train[0]), len(x_train[0][0]), len(x_train[0][0]), "\n", ("=")*30))

# print("This is whole:", "\n", ("-")*30, "\n", x_train, "\n", ("=")*30)
# print("This is lev 1:", " \n", ("-")*30, "\n", x_train[0], "\n", ("=")*30)
# print("This is lev 2:", " \n", ("-")*30, "\n", x_train[0][0], "\n", ("=")*30)
# print("This is lev 3:", " \n", ("-")*30, "\n", x_train[0][0][0], "\n", ("=")*30)

In [5]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

template1 ='whole: {} | lev 1: {} | lev 2: {} | lev 3: {}'
# print(template1.format(len(x_train), len(x_train[0]), len(x_train[0][0]), len(x_train[0][0][0]), "\n", ("-")*30))

# print("This is whole:", "\n", ("-")*30, "\n", x_train, "\n", ("=")*30)
# print("This is lev 1:", " \n", ("-")*30, "\n", x_train[0], "\n", ("=")*30)
# print("This is lev 2:", " \n", ("-")*30, "\n", x_train[0][0], "\n", ("=")*30)
# print("This is lev 3:", " \n", ("-")*30, "\n", x_train[0][0][0], "\n", ("=")*30)
# print("This is lev 4:", " \n", ("-")*30, "\n", x_train[0][0][0][0], "\n", ("=")*30)

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [7]:
# print(train_ds)
# print(test_ds)

In [8]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [11]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

In [12]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [13]:
def train_model(epochs):
    startTime = time.time()
    for epoch in range(epochs):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
#             train_step(images, labels)
#             to suppress the float warning, a better code is below , Henry-Jul. 2, 2020
            train_step(tf.cast(images, tf.float32), labels)

        for test_images, test_labels in test_ds:
#             test_step(test_images, test_labels)
            test_step(tf.cast(test_images, tf.float32), test_labels)

        template = 'Epoch {} :: Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print(template.format(epoch + 1,
                                train_loss.result(),
                                train_accuracy.result() * 100,
                                test_loss.result(),
                                test_accuracy.result() * 100))
        
    total_run_time = time.time() - startTime
    return total_run_time

### Set epochs and iterations

In [14]:
epochs = 30
iteration = 3 
run_time = []

In [15]:
# break-here # on purpose so the cpu kernel upon switching won't run & override the benchmark below

### Kernel :: tf-cpu-env  :: using CPU

In [ ]:
# with cpu
for i in range(iteration):
    print(f'{("-")*30}\n Iteration # {i + 1}')
    run_time.append(train_model(epochs))
    
for i in range(iteration):
    print(f'{("-")*30}\nCPU Run Summary ::\n Run # {i + 1} ::\
        {round(run_time[i]/60, 3)} {"minutes" if run_time[i]/60 > 1 else "minute"}')

------------------------------
 Iteration # 1
Epoch 1 :: Loss: 0.1364217847585678, Accuracy: 95.96833801269531, Test Loss: 0.05502424016594887, Test Accuracy: 98.08999633789062
Epoch 2 :: Loss: 0.04312481731176376, Accuracy: 98.64166259765625, Test Loss: 0.04803283512592316, Test Accuracy: 98.3499984741211
Epoch 3 :: Loss: 0.02306361496448517, Accuracy: 99.25, Test Loss: 0.049662452191114426, Test Accuracy: 98.38999938964844
Epoch 4 :: Loss: 0.013225630857050419, Accuracy: 99.55999755859375, Test Loss: 0.054637085646390915, Test Accuracy: 98.43999481201172
Epoch 5 :: Loss: 0.009830133058130741, Accuracy: 99.66333770751953, Test Loss: 0.05781836062669754, Test Accuracy: 98.4000015258789
Epoch 6 :: Loss: 0.0066680097952485085, Accuracy: 99.77833557128906, Test Loss: 0.059936296194791794, Test Accuracy: 98.54000091552734
Epoch 7 :: Loss: 0.00579359894618392, Accuracy: 99.79666137695312, Test Loss: 0.08149466663599014, Test Accuracy: 98.15999603271484
Epoch 8 :: Loss: 0.0062210094183683395

In [17]:
break-here # on purpose so the cpu kernel upon switching won't run & override the benchmark below

SyntaxError: invalid syntax (<ipython-input-17-f242f9b63921>, line 1)

### Kernel :: tf-gpu-env  :: using GPU

In [15]:
# with cpu
for i in range(iteration):
    print(f'{("-")*30}\n Iteration # {i + 1}')
    run_time.append(train_model(epochs))
    
for i in range(iteration):
    print(f'{("-")*30}\nGPU Run Summary ::\n Run # {i + 1} ::\
        {round(run_time[i]/60, 3)} {"minutes" if run_time[i]/60 > 1 else "minute"}')

------------------------------
 Iteration # 1
Epoch 1 :: Loss: 0.13836054503917694, Accuracy: 95.87332916259766, Test Loss: 0.06307229399681091, Test Accuracy: 97.95999908447266
Epoch 2 :: Loss: 0.042076319456100464, Accuracy: 98.75, Test Loss: 0.05329422280192375, Test Accuracy: 98.4000015258789
Epoch 3 :: Loss: 0.020792454481124878, Accuracy: 99.3499984741211, Test Loss: 0.0670895129442215, Test Accuracy: 98.15999603271484
Epoch 4 :: Loss: 0.013897608034312725, Accuracy: 99.5250015258789, Test Loss: 0.06296999752521515, Test Accuracy: 98.30999755859375
Epoch 5 :: Loss: 0.010262991301715374, Accuracy: 99.63999938964844, Test Loss: 0.0723794624209404, Test Accuracy: 98.1500015258789
Epoch 6 :: Loss: 0.006475887261331081, Accuracy: 99.79166412353516, Test Loss: 0.07033147662878036, Test Accuracy: 98.3699951171875
Epoch 7 :: Loss: 0.00624010618776083, Accuracy: 99.78333282470703, Test Loss: 0.07001430541276932, Test Accuracy: 98.37999725341797
Epoch 8 :: Loss: 0.003922380972653627, Accur

Epoch 5 :: Loss: 5.563099633731916e-11, Accuracy: 100.0, Test Loss: 0.17263054847717285, Test Accuracy: 98.5999984741211
Epoch 6 :: Loss: 5.960464066756543e-11, Accuracy: 100.0, Test Loss: 0.17398644983768463, Test Accuracy: 98.6199951171875
Epoch 7 :: Loss: 5.563099633731916e-11, Accuracy: 100.0, Test Loss: 0.17432323098182678, Test Accuracy: 98.62999725341797
Epoch 8 :: Loss: 4.967053504612018e-11, Accuracy: 100.0, Test Loss: 0.1751563549041748, Test Accuracy: 98.6199951171875
Epoch 9 :: Loss: 3.774960552482831e-11, Accuracy: 100.0, Test Loss: 0.17534828186035156, Test Accuracy: 98.5999984741211
Epoch 10 :: Loss: 4.967053504612018e-11, Accuracy: 100.0, Test Loss: 0.1767193078994751, Test Accuracy: 98.58999633789062
Epoch 11 :: Loss: 4.172324638562763e-11, Accuracy: 100.0, Test Loss: 0.17688961327075958, Test Accuracy: 98.61000061035156
Epoch 12 :: Loss: 3.973642595522797e-11, Accuracy: 100.0, Test Loss: 0.17850342392921448, Test Accuracy: 98.58000183105469
Epoch 13 :: Loss: 3.5762781